# CASSANDRA DEMOS

# Nodetool

Nodetool es una herramienta de administración que permite una amplia gama de operaciones tanto sobre nodos como sobre el cluster de Cassandra. La podremos encontrar en el directorio /bin de la instalación. A continuación, realizaremos un recorrido por algunas de las funcionalidades que nos ofrece esta utilidad. Para ello, comenzaremos abriendo la terminal.

* Si queremos obtener la versión de Cassandra que se está ejecutando en el nodo podremos hacerlo ejecutando:<br><br><b><i>$ nodetool version</i></b><br><br>En nuestro caso la salida a dicho comando será:<br><br><b><i>Release Version: 3.11.4</i></b>

* Para obtener el estado de los nodos del anillo podemos ejecutar el comando:<br><br><b><i>$ nodetool status</b></i><br><br>El cluster instalado dispone de un único nodo que, por tanto, será el responsable de todos los tokens. El estado de dicho nodo será <b>UN</b> que hace referencia a Up y Normal. Una forma de buscar nodos que estén caídos puede ser mediante: <br><br><b><i>\$ nodetool status | grep DN</i></b>

* Para seguir profundizando, podemos obtener información de los tokens del nodo de la siguiente forma:<br><br><b><i> $ nodetool ring </i></b>

* Si se desean obtener estadísticas del nodo como los segundos que lleva activo, el estado del Gossip, el estado del CQL Native Transport que es la forma en la que el driver de Cassandra se comunica con el server,  el número de excepciones, uso de memoria heap o de distintas caches, entre otra información: <br><br><b><i> $ nodetool info </i></b>

* Mediante el siguiente comando se puede obtener información relacionada con la latencia de las escrituras, de las lecturas, el espacio total de disco utilizado en cada keyspace: <br><br><b><i> $ nodetool tablestats</i></b>

* Empleando el flag -h podremos ver todas las opciones que ofrece nodetool:<br><br><b><i> $ nodetool -h </i></b>

* Además si queremos ver flags específicos relacionados con algún comando en concreto de nodetool (por ejemplo de status) ejecutaremos:<br><br><b><i> $ nodetool help status </i></b>

# CQLSH 

En este apartado vamos a explorar la shell de CQL (siglas de Cassandra Query Language) que tiene bastantes similitudes en la sintaxis con SQL. Gracias a esta shell podremos empezar a utilizar el lengueje CQL para manipular información empleando el modelo de datos de Cassandra.

### Login

El primero paso es abrir una terminal y ejecutar: <br><br><b><i> $ cqlsh </i></b>

Se obtendrá un prompt donde podremos escribir las sentencias de CQL. Dicho prompt tendrá el siguiente aspecto:<br><br> <b><i>cassandra@cqlsh></i></b>

Vamos a salir de CQLSH escribiendo el comando: <b><i>exit</i></b>

Una vez que hayamos vuelto a la shell de la terminal de Ubuntu vamos a editar el fichero de configuración cassandra.yaml mediante el comando:<br><br><b><i>$ sudo gedit /etc/cassandra/cassandra.yaml</i></b>

Comentaremos la línea:<br><br><b><i>#authenticator: AllowAllAuthenticator </i></b><br><br>

Y añadiremos la línea y guardaremos los cambios:<br><br><b><i>authenticator: PasswordAuthenticator</i></b>

Ese cambio provocará que sea necesario especificar el nombre de usuario y su contraseña que estarán almacenados en la tabla system_auth.roles estando las contraseñas hasheadas por temas de seguridad.

Para que el cambio se haga efectivo tendremos que reiniciar el servicio de Cassandra ejecutando:<br><br><b><i> $ sudo service cassandra restart</i></b>

Si intentamos ejecutar CQLSH de la misma forma que antes obtendremos un error en el que se nos indicará que debemos especificar las credenciales de un usuario válido. Emplearemos el usuario por defecto 'cassandra' con su contraseña 'cassandra':<br><br><b><i>$ cqlsh localhost -u cassandra -p cassandra</i></b>

Una vez hecho eso estaremos de nuevo dentro del prompt de cassandra (cassandra@cqlsh>) donde podremos ejecutar sentencias CQL de forma que cada una de ellas debe tener en <b>;</b>

Para tener una visión general del conjunto de sentencias disponibles ejecutaremos el comando:<br><br><b><i>cassandra@cqlsh> help;</i></b>

Como hemos hemos empleado el usuario 'cassandra' para conectarnos al cluster, vamos a ver qué sentencia tendremos que ejecutar para comprobar los usurios disponibles (en este punto se comprobará que sólo se dispone del usuario 'cassandra'):<br><br><b><i>cassandra@cqlsh> list users;</i></b>

La salida se muestra en forma de tabla con 2 columnas: name y super cuyos valores son 'cassandra' y 'yes' respectivamente.

Para crear un nuevo usuario llamado 'test_1' que no sea superusuario se tendría que emplear la siguiente sentencia:<br><br><b><i>cassandra@cqlsh> CREATE USER IF NOT EXISTS test_1 WITH PASSWORD 'test_1' NOSUPERUSER;</i></b> 

Volveremos a ejecutar el comando: <br><br><b><i>cassandra@cqlsh> list users;</i></b>

Si se quiere cambiar la contraseña del usuario test_1 y hacerle superusuario tendremos que ejecutar:<br><br><b><i>cassandra@cqlsh> ALTER USER test_1 WITH PASSWORD 'test' SUPERUSER;

Para comprobar los cambios realizados podemos tratar de loguearnos en cqlsh como el usuario test_1 y ejecutar list users;

La sentencia necesaria para cambiar de usuario sin salir de cqlsh es:<br><br><b><i>cassandra@cqlsh> login test_1;</i></b>

Al ejecutarla Cassandra nos pedirá que introduzcamos la contraseña de ese usuario y una vez hecho eso podremos comprobar como el prompt cambia a:<br><br><b><i>test_1@cqlsh></i></b>

Es posible borrar un usuario existente desde una cuenta de superusuario(como cassandra) empleando la sentencia:<br><br><b><i>cassandra@cqlsh> DROP USER IF EXISTS test_1;</i></b>

Hay que señalar que un usuario no puede borrarse a sí mismo.

### Operaciones CRUD (Create, Read, Update, Delete)

El siguiente paso que realizaremos será crear un keyspace que, como dijimos, puede tratar de verse como una BBDD en un sistema relacional. Como nuestro cluster está formado por un único server, emplearemos como estrategia de replicación 'SimpleStrategy' y con factor de replicación 1:<br><br><b><i> cassandra@cqlsh> CREATE KEYSPACE clientes WITH REPLICATION = {'class': 'SimpleStrategy', 'replication_factor': 1};</i></b>

Podemos consultar los keyspaces disponibles mediante:<br><br><b><i> cassandra@cqlsh> DESCRIBE keyspaces;</i></b>

Para trabajar con un keyspace en concreto, por ejemplo con 'clientes', tendremos que ejecutar:<br><br><b><i>cassandra@cqlsh> USE clientes;</i></b>

Una vez hecho eso, veremos que el prompt pasa a ser <b><i>cassandra@cqlsh:clientes> </i></b> y se pueden comprobar las tablas presentes en ese keyspace (ya se adelanta que no habrá ninguna) mediante el comando:<br><br><b><i>cassandra@cqlsh:clientes> DESCRIBE TABLES;</i></b>

Vamos a crear una tabla sencilla llamada 'cuenta_smart' que tendrá las columnas 'nombre', 'fecha_creacion', 'saldo' y 'edad' en la que se empleará como partition key el nombre del cliente:<br><br><b><i>cassandra@cqlsh:clientes> CREATE TABLE cuenta_smart(nombre text, fecha_creacion text, saldo decimal, edad int, primary key(nombre));</i></b>

Si volvemos a comprobar las tablas disponibles en el keyspace ya debe aparecer la descripción de 'cuenta_smart':<br><br><b><i>cassandra@cqlsh:clientes> DESCRIBE TABLES;</i></b>

La inserción de una fila se realizaría de la siguiente forma:<br><br><b><i> cassandra@cqlsh:clientes> INSERT INTO cuenta_smart(nombre, fecha_creacion, saldo, edad) VALUES ('Alberto Ríos', '2019-03-05', 5432.21, 34);</i></b>

Se puede consultar su contenido mediante:<br><br><b><i>cassandra@cqlsh:clientes> SELECT nombre, fecha_creacion, saldo, edad FROM cuenta_smart;</i></b>

La sentencia necesaria para actualizar el valor de alguna columna, por ejemplo, de la columna saldo para el caso del nombre 'Alberto Ríos' es:<br><br><b><i>cassandra@cqlsh:clientes> UPDATE cuenta_smart SET saldo = 1000 WHERE nombre = 'Alberto Ríos';</i></b>

Podemos borrar valores de columnas de una tabla mendiante la cláusula <b><i>DELETE</i></b>. Por ejemplo, si se desea borrar el saldo del usuario 'Alberto Ríos':<br><br><b><i>cassandra@cqlsh:clientes> DELETE saldo FROM cuenta_smart WHERE nombre = 'Alberto Ríos';</i></b>

Si se realiza una consulta del saldo de 'Alberto Ríos' se obtendrá el valor null.

### Map e INDEX

#### Map

Cassandra permite conseguir una aproximación a la características de schema-less (que significa que cada fila de una column family puede contener un número diferente de columnas que no tienen por qué haber sido declaradas previamente) empleando un Map a partir de Cassandra 2.1. Vamos a ver un ejemplo:<br><br><b><i> cassandra@cqlsh:clientes> CREATE TABLE cuenta_123(id text, nombre text, movimiento map<text, text>, primary key(id));</i></b>

Ahora vamos a añadir una serie de filas de datos: <br><br><b><i> cassandra@cqlsh:clientes> INSERT INTO cuenta_123 (id, nombre, movimiento) VALUES ('1', 'Juan Pérez', {'tipo': 'transferencia nacional', 'concepto': 'Pago hipoteca'});</i></b> 

<b><i>cassandra@cqlsh:clientes> INSERT INTO cuenta_123 (id, nombre, movimiento) VALUES ('2', 'Juan Pérez', {'tipo': 'ingreso', 'cantidad': '2000', 'fecha': '2019-02-14'});</i></b> 

<b><i>cassandra@cqlsh:clientes> INSERT INTO cuenta_123 (id, nombre, movimiento) VALUES ('3', 'María González', {'tipo': 'consulta_saldo'});</i></b>

Si se hace una consulta se comprobará que la tabla tiene 3 columnas llamadas 'id', 'nombre' y 'movimiento' de forma que esta última tiene una estrucutura de pares clave-valor que puede ser distinto en cada fila pero en la definición se establece que las claves tienen que ser de tipo 'text' y el valor en este caso también se ha definido de tipo 'text'. Por tanto, hay que respetar los tipos establecidos en la definición aunque puede variar el número y el contenido de los pares clave-valor de la columna 'movimiento':<br><br><b><i>cassandra@cqlsh:clientes> SELECT * FROM cuenta_123;</i></b> 

#### Create Index

Ahora vamos a crear un índice para las claves de la columna 'movimiento' y otro para la columna 'nombre'. Se pueden crear índices para claves de colecciones y para sus valores, pero no para los 2 simultáneamente. Para ello, emplearemos la cláusula <b><i>CREATE INDEX</i></b> que permite definir un índice para una columna a excepción por ejemplo de la partition key. En caso de que la columna que se quiera indexar ya tenga datos, Cassandra asignará índices a esos datos durante la ejecución de la sentencia CREATE INDEX. Cuando se inserten nuevos datos, serán indexados automáticamente. Si una columna no dispone de índices, Cassandra no podrá filtrar empleando dicha columna a no ser que se trate de la partition key.<br><br><b><i> cassandra@cqlsh:clientes> CREATE INDEX indmov ON cuenta_123 (KEYS(movimiento));</i></b>

<b><i>cassandra@cqlsh:clientes> CREATE INDEX indmove ON cuenta_123(ENTRIES(movimiento));</i></b>

<b><i>cassandra@cqlsh:clientes> CREATE INDEX indmovv ON cuenta_123(VALUES(movimiento));</i></b>

<b><i>cassandra@cqlsh:clientes> CREATE INDEX ON cuenta_123 (nombre);</i></b>

En caso de no especificar el nombre del índice, Cassandra le asignará uno empleando el formato <i>nombretabla_nombrecolumna_idx</i>. Si el nombre que se quiere asignar al indentificador del índice contiene caracteres especiales o si se quiere que se respete el uso mayúsculas y minúsculas habrá que poner el nombre del índice entre comillas. Una vez hecho esto, ya podremos realizar consultas empleando las claves del Map, sus valores o la columna 'nombre':<br><br><b><i> cassandra@cqlsh:clientes> SELECT * FROM cuenta_123 WHERE movimiento CONTAINS KEY 'concepto';</i></b>

<b><i>cassandra@cqlsh:clientes> SELECT * FROM cuenta_123 WHERE moviento['cantidad']= '2000';</i></b>

<b><i>cassandra@cqlsh:clientes> SELECT * FROM cuenta_123 WHERE movimiento CONTAINS 'transferencia nacional';</i></b>

<b><i>cassandra@cqlsh:clientes> SELECT * FROM cuenta_123 WHERE nombre = 'Juan Pérez';</i></b>

### Composite Partition Key

Cuando se define una tabla se establece cuál es su primary key que puede estar formada por una o varias columnas. Si dicha <b>clave primaria está formada por una única columna</b>, esa será la partition key, que como se comentó anteriormente, cuando se vaya a insertar una fila se realizará el hash de su partition key y el resultado del hash (conocido como token) se empleará para determinar qué nodo almacenará esa fila sabiendo que cada nodo del anillo almacena un rango de tokens. 

Si la <b>primary key está formada por varias columnas con un único nivel de paréntesis</b> (<b><i>PRIMARY KEY (col_A, col_b, col_C)</i></b>) entonces la primera columna de dicha lista será la que actúe como partition key y las columnas restantes formarán lo que se conoce como <b>clustering keys</b>, de las que hablaremos en breve, pero como introducción avanzaremos diciendo que permiten ordenar los datos ascendente o descendentemente dentro de las particiones.

En caso de que la <b>primary key esté formada por varias columnas con 2 niveles de paréntesis</b> (<b>PRIMARY KEY ((col_A, col_C), col_B)</b>) se denominará <b>composite partition key</b> a la tupla (col_A, col_C) y será la que se empleará para determinar en qué nodo se encuentra una fila. La columna llamada col_B se corresponderá con la clustering key.

Como ejemplo vamos a crear una tabla que tenga una composite partition key, insertaremos filas y realizaremos una consulta sobre esa tabla.

<b><i>cassandra@cqlsh:clientes> CREATE TABLE cuenta_zero123 (nombre text, email text, mov_dt date, saldo double, PRIMARY KEY ((nombre, email), mov_dt));</i></b>

<b><i>cassandra@cqlsh:clientes> INSERT INTO cuenta_zero123 (nombre, email, mov_dt, saldo) VALUES ('José Ramírez', 'jr@gmail.com', '2019-04-23', 6040.24);<br><br>cassandra@cqlsh:clientes> INSERT INTO cuenta_zero123(nombre, email, mov_dt, saldo) VALUES ('Silvia Pérez', 'sp@gmail.com', '2019-04-23', 12456.48);<br><br>cassandra@cqlsh:clientes> INSERT INTO cuenta_zero123(nombre, email, mov_dt, saldo) VALUES ('Raquel Fernández', 'rf@gmail.com', '2019-04-24', 5762.34);
<br><br>cassandra@cqlsh:clientes> INSERT INTO cuenta_zero123(nombre, email, mov_dt, saldo) VALUES ('Alberto Martín', 'am@gmail.com', '2019-04-25', 3762.34);
<br><br>cassandra@cqlsh:clientes> INSERT INTO cuenta_zero123(nombre, email, mov_dt, saldo) VALUES ('Alberto Martín', 'am@gmail.com', '2019-04-30', 3752.36);
</i></b>

<b><i>cassandra@cqlsh:clientes> SELECT * FROM cuenta_zero123 WHERE nombre = 'Silvia Pérez' and email = 'sp@gmail.com';</i></b>

La última consulta devolverá la fila asociada a Silvia Pérez con email sp@gmail.com. 

### Allow Filtering

Si, por ejemplo, se quisiera realizar una consulta para obtener las filas de la tabla cuenta_zero123 en las que el campo saldo toma un determinado valor (en este caso, se buscará el valor 6040.24) se puede comprobar que Cassandra devolverá un error. Hay que recordar que aunque en nuestro escenario de pruebas sólo disponemos de 1 nodo, en un entorno real el cluster puede contar con multitud de nodos y en la consulta se está empleando para filtrar el campo saldo que no forma parte de la partition key. No se dispone de información para saber en qué partición y por tanto en qué nodo se encuentra la información buscada. Como nos encontramos en un entorno que puede ser de Big Data, el realizar una búsqueda masiva por todo el cluster resultará ineficiente.  

Vamos a comprobar qué ocurre cuando se trata de ejecutar esta consulta:<br><br><b><i>cassandra@cqlsh:clientes> SELECT * FROM cuenta_zero123 WHERE saldo = 6040.24;</i></b>

De todas formas, pensando en entornos de desarrollo y test, Cassandra permite ejecutar este tipo de consultas en las que se emplea como campo de filtrado alguna columna que no forma parte de la partition key especificando en la propia consulta la cláusula <b>Allow Filtering</b>.

Vamos a modificar la consulta anterior añadiendo la cláusula Allow Filtering:<br><br><b><i>cassandra@cqlsh:clientes> SELECT * FROM cuenta_zero123 WHERE saldo = 6040.24 ALLOW FILTERING;</i></b>

### Clustering Key

Las clustering keys serán aquellas columnas de la primary key que no forman parte de la partition key o composite partition key y que permiten introducir un orden dentro de cada partición (no globalmente). Vamos a comprobar la definición de la tabla cuenta_zero123:<br><br><b><i>cassandra@cqlsh:clientes> DESCRIBE cuenta_zero123;</i></b>

Por defecto introduce un orden ascendente en disco dentro de cada partición pero se puede indicar que dicho orden sea descendente. Cuando realicemos una consulta sin especificar en la cláusula WHERE la partition key, el resultado estará ordenado en función de los valores hash de la partition key.

<b><i>cassandra@cqlsh:clientes> SELECT * FROM cuenta_zero123;</i></b>

Si queremos obtener los valores de los hashes de la composite partiton key para entender el orden obtenido en la consulta anterior podemos llamar a la función token sobre dichas claves:<br><br><b><i>cassandra@cqlsh:clientes> SELECT token(nombre, email), nombre, email FROM cuenta_zero123;</i></b>

Si especificamos un valor de partition key en la cláusula WHERE, los resultados sí se delvolverán ordenados en función de la clustering key dentro de la partición.<br><br><b><i> cassandra@cqlsh:clientes> SELECT * FROM cuenta_zero123 WHERE nombre = 'Alberto Martín' AND email = 'am@gmail.com';

Vamos a crear otra tabla en la que habrá varios clustering keys para comprobar que el orden en que se definan establecerá la secuencia de prioridad (de izquieda a derecha) a la hora de realizar la ordenación dentro de cada partición.

<b><i>cassandra@cqlsh:clientes> CREATE TABLE cuenta_zero1234 (nombre text, email text, mov_dt date, saldo decimal, PRIMARY KEY((nombre, email), mov_dt, saldo));<br><br>cassandra@cqlsh:clientes> INSERT INTO cuenta_zero1234 (nombre, email, mov_dt, saldo) VALUES ('Sara Romero', 'sar@gmail.com', '2019-06-02', 2400);<br><br>cassandra@cqlsh:clientes> INSERT INTO cuenta_zero1234 (nombre, email, mov_dt, saldo) VALUES ('Roberto Serra', 'rose@gmail.com', '2019-07-24', 3200);<br><br>cassandra@cqlsh:clientes> INSERT INTO cuenta_zero1234 (nombre, email, mov_dt, saldo) VALUES ('Roberto Serra', 'rose@gmail.com', '2019-08-21', 4000);<br><br>cassandra@cqlsh:clientes> INSERT INTO cuenta_zero1234 (nombre, email, mov_dt, saldo) VALUES ('Roberto Serra', 'rose@gmail.com', '2019-08-26', 2700);<br><br>cassandra@cqlsh:clientes> INSERT INTO cuenta_zero1234 (nombre, email, mov_dt, saldo) VALUES ('Roberto Serra', 'rose@gmail.com', '2019-08-26', 2300);
</i></b>

<b><i>cassandra@cqlsh:clientes> SELECT * FROM cuenta_zero1234 WHERE nombre = 'Roberto Serra' AND email = 'rose@gmail.com';</i></b>

Se comprueba que el orden de presentación de las filas es el siguiente:<br><br>'Roberto Serra', 'rose@gmail.com', '2019-07-24', 3200<br><br>'Roberto Serra', 'rose@gmail.com', '2019-08-21', 4000<br><br>'Roberto Serra', 'rose@gmail.com', '2019-08-26', 2300<br><br>'Roberto Serra', 'rose@gmail.com', '2019-08-26', 2700


Como se comentó anteriormente, las clustering keys establecen por defecto un orden ascendente dentro de cada partición pero se puede especificar que el orden sea descendente durante la creación de la tabla empleando la cláusula <b><i>WITH CLUSTERING ORDER BY</i></b>:

<b><i>cassandra@cqlsh:clientes> CREATE TABLE cuenta_zero12345 (nombre text, email text, mov_dt date, saldo decimal, PRIMARY KEY ((nombre, email), mov_dt, saldo)) WITH CLUSTERING ORDER BY (mov_dt DESC, saldo ASC);</i></b>

<b><i>cassandra@cqlsh> INSERT INTO cuenta_zero1234 (nombre, email, mov_dt, saldo) VALUES ('Roberto Serra', 'rose@gmail.com', '2019-07-24', 3200);</i></b>

<b><i>cassandra@cqlsh:clientes> INSERT INTO cuenta_zero1234 (nombre, email, mov_dt, saldo) VALUES ('Roberto Serra', 'rose@gmail.com', '2019-08-21', 4000);</i></b>

<b><i>cassandra@cqlsh:clientes> INSERT INTO cuenta_zero1234 (nombre, email, mov_dt, saldo) VALUES ('Roberto Serra', 'rose@gmail.com', '2019-08-26', 2700);</i></b>

<b><i>cassandra@cqlsh:clientes> INSERT INTO cuenta_zero1234 (nombre, email, mov_dt, saldo) VALUES ('Roberto Serra', 'rose@gmail.com', '2019-08-26', 2300);</i></b>

Los resultados de las consultas estarán ordenados en cada partición primero descendentemente en función de mov_dt y después ascendentemente teniendo en cuenta el saldo.

<b><i>cassandra@cqlsh:clientes> SELECT * FROM cuenta_zero12345 WHERE nombre = 'Roberto Serra' AND email = 'rose@gmail.com';</i></b>

A la hora de elegir partition keys y clustering keys hay que tener en cuenta algunos factores para que realmente sean de utilidad:

* Evitar particiones demasiado grandes. 
* Evitar emplear como primary key campos que para algunos de sus valores creen hotspots intentando elegir aquellos que distribuyan lo más homogéneamente los datos. 

### Materialized View

Vamos a crear la tabla <b>clien</b>:<br><br><b><i>cassandra@cqlsh:clientes> CREATE TABLE clien (cod text, nombre text, dni text, edad int, cuentas list \< text \>, PRIMARY KEY(cod));</i></b>

Insertaremos algunos registros:

<b><i>cassandra@cqlsh> INSERT INTO clien (cod, nombre, dni, edad, cuentas) VALUES ('a21', 'Roberto Serra', '03374591J', 46, ['cuenta zero1234']);</i></b>

<b><i>cassandra@cqlsh> INSERT INTO clien (cod, nombre, dni, edad, cuentas) VALUES ('a34', 'Sara Romero', '04274991C', 34, ['cuenta zero1234']);</i></b>

<b><i>cassandra@cqlsh> INSERT INTO clien (cod, nombre, dni, edad, cuentas) VALUES ('a26', 'Mario Caballero', '04678591V', 34, ['cuenta zero1234', 'cuenta 123']);</i></b>

<b><i>cassandra@cqlsh> INSERT INTO clien (cod, nombre, dni, edad, cuentas) VALUES ('a29', 'Silvia Aparicio', '07674591U', 46,  ['cuenta zero123']);</i></b>

Si quisiéramos realizar consultas sobre esa tabla empleando como campo de búsqueda la edad podríamos crear un index como haríamos con una base de datos relacional:

<b><i>cassandra@cqlsh> CREATE INDEX indedad ON clien(edad);</i></b>

El problema es que como el index no es el partition key, Cassandra no sabrá dónde buscar la edad deseada y a medida que el cluster vaya teniendo más nodos, Cassandra tendrá que preguntar a todos ellos y reunir todas las respuestas.

La forma de trabajar en Cassandra es con datos desnormalizados. Como las escrituras en Cassandra no son costosas, se puede pensar en duplicar la información donde sea preciso para que a la hora de realizar las consultas, dicha información pueda estar en una misma tabla accesible mediante partition key y ordenada dentro de cada partición por clustering key.

Para disponer de la información de <b>clien</b> accesible mediante edad se puede crear otra tabla de la siguiente forma:

<b><i>cassandra@cqlsh:clientes> CREATE TABLE clien_edad (cod text, nombre text, dni text, edad int, cuentas list \< text \> PRIMARY KEY(edad));</i></b>

En este punto se podrán realizar consultas del tipo:

<b><i>cassandra@cqlsh:clientes> SELECT * FROM clien_edad WHERE edad = 34;</i></b>

La parte tediosa será que a la hora de realizar inserciones, actualizaciones o borrados en alguna de las 2 tablas también habrá que hacerlo en la otra para mantener la consistencia.

Aquí es donde entran en juego las <b>Materialized Views</b>. Se trata de tablas que son gestionadas por Cassandra de forma autónoma en vez de tener que crear manualmente tablas desnormalizadas con información duplicada y que haya que mantener sincronizadas. La idea es partir de una tabla origen y a partir de ella generar una Materialized View inspirada en la original pero en la que se alterarán y / o modificarán el orden de las primary keys.

A continuación, vamos a ver una serie de restricciones a tener en cuenta para poder emplear Materialized Views:

* Todas las claves que aparezcan en la primary key de la tabla original, es decir, partiton keys y clustering keys deben estar presentes en la primary key de la Materialized View aunque pueden estar en orden diferente.

* La clave primaria de la Materialized View sólo puede tener una columna que no aparezca en la clave primaria de la tabla original (es difícil para Cassandra manejar la consistencia cuando están involucradas actualizaciones concurrentes con columnas que no aparecen en la primary key de la tabla base).

* Las columnas que vayan a formar parte de la primary key en la Materialized View deben definirse con la restricción <b>IS NOT NULL</b> para evitar que haya registros en los que no se disponga de datos para las columnas de la primary key.

* Tiene un coste de procesamiento ya que realizar una escritura en la tabla original implica bloquear la partición, leer el contenido de la misma (el bloqueo se realiza para evitar que se puedan realizar 2 escrituras simultáneas antes de que se haya replicado de forma consistente la información en la MV), calcular los cambios a realizar en la MV, crear un batch con los cammbios a realizar en la tabla original y en la MV para asegurar atomicidad (de forma que si la escritura en la tabla base es correcta eso signifique que en la MV también se realice la escritura) y ejecutar los cambios en la tabla original y en la MV.

Vamos a crear una Materialized View llamada <b>cli_por_edad_mv</b> de forma que se puedan realizar búsquedas en función de la edad del cliente teniendo en cuenta las restricciones anteriores:

<b><i>cassandra@cqlsh:clientes> CREATE MATERIALIZED VIEW cli_por_edad_mv AS SELECT cod, nombre, dni, edad, cuentas FROM clien WHERE cod IS NOT NULL AND edad IS NOT NULL PRIMARY KEY (edad, cod);</i></b> 

Se obtendrá el mensaje:

<b><i>Warnings :
    Materialized views are experimental and are not recommended for production use.</i></b>

Podremos realizar la consulta:

<b><i>cassandra@cqlsh:clientes> SELECT * FROM cli_por_edad_mv WHERE edad = 34;</i></b>

Si por ejemplo se realiza una inserción en la tabla base <b>clien</b> comprobaremos si la MV <b>cli_por_edad_mv</b> refloja automáticamente esa inserción:

<b><i>cassandra@cqlsh:clientes> INSERT INTO clien (cod, nombre, dni, edad, cuentas) VALUES ('a56', 'Antonio Ramos', '05274291V', 34, ['cuenta zero1234', 'cuenta 123']);</i></b>

Comprobaremos tanto el contenido de la tabla <b>clien</b> como de la MV <b>cli_por_edad_mv</b> para verificar que en ambas se ha realizado la inserción:

<b><i>cassandra@cqlsh:clientes> SELECT * FROM clien;</i></b>

<b><i>cassandra@cqlsh:clientes> SELECT * FROM cli_por_edad_mv;</i></b>